In [1]:
'''
基于PointNet++的CNN模型，未完成batch维度。
输入是预备体的点云和评分，输出是训练好的神经网络，可用于对新预备体点云评分。
尚有优化余地。此外，接下来还可以试一下算量更大的体素模型，那个模型便于同时输入真牙和预备体的图像。
'''

'\n基于PointNet++的CNN模型，能够实现基本功能。没有为提高运算效率而牺牲可读性，包会。\n输入是预备体的点云和评分，输出是训练好的神经网络，可用于对新预备体点云评分。\n尚有优化余地。此外，接下来还可以试一下算量更大的体素模型，那个模型便于同时输入真牙和预备体的图像。\n'

In [2]:
'''
# 安装环境，和上次发的代码一样的操作方式
!pip install --upgrade tensorflow
'''

'\n# 安装环境，和上次发的代码一样的操作方式\n!pip install --upgrade tensorflow\n'

In [3]:
'''
# 更新环境，以解决numpy和tensorflow的兼容性问题
pip install --upgrade numpy tensorflow
'''

'\n# 更新环境，以解决numpy和tensorflow的兼容性问题\npip install --upgrade numpy tensorflow\n'

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

2024-07-04 10:02:35.193634: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 10:02:35.200825: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 10:02:35.215514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 10:02:35.251579: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 10:02:35.251627: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 10:02:35.276037: I tensorflow/core/platform/cpu_feature_guard.cc:

In [5]:
# 加载输入信息

def read_scores(score_file_path):
    # 读全部牙的序号和评分
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores

def read_obj_file(obj_file_path):
    # 读单颗牙的3D模型
    points = []
    with open(obj_file_path, 'r') as f:
        for line in f:
            if line.startswith('v '):
                _, x, y, z = line.strip().split()
                points.append([float(x), float(y), float(z)])
    return np.array(points)


In [6]:
# Set Abstraction模块，用于提取点云的局部特征

def sampling_layer(points, npoints):
    '''
    最远点采样 Farthest Point Sample，通过“让采样点之间的最小距离最大化”来用尽可能少的点代表尽可能多的特征。
    
    算法是：
        （1）先随机选取一个点，计算其余点到这个点的距离，选取距离最大的点做第二个点；
        （2）已采样N个点时，计算所有剩余点到每个已采样点的距离，对单个剩余点总有一个已采样点在采样点中距离此剩余点最近，取前述最近距离最大的剩余点做为第N+1个采样点；
        （3）重复此过程直至达到采样数目要求。
    
    输入是：
        点云（每个点的前三个维度是其空间坐标，其余维度是特征）、采样数。
    输出是：
        采样出的点云（空间坐标维度+特征维度）。
    '''
    # 确认点数和采样数
    N, D = points.shape
    if npoints > N:
        # 如果采样点数设置得比点云点数还大，自动报错
        raise ValueError("npoints should be less than or equal to the number of points in the point cloud")

    # 初始化返回的采样点数组
    sampled_points = np.zeros((npoints, D))
    
    # 初始化距离数组，设置为一个较大的数值
    distances = np.ones(N) * 1e10
    
    # 随机选择第一个点
    first_point_index = np.random.randint(0, N)
    sampled_points[0] = points[first_point_index]
    
    # 更新所有点到第一个采样点的距离
    for i in range(N):
        distances[i] = np.linalg.norm(points[i, :3] - points[first_point_index, :3])
    
    for i in range(1, npoints):
        # 选择距离当前所有采样点最远的点
        farthest_point_index = np.argmax(distances)
        sampled_points[i] = points[farthest_point_index]
        
        # 更新所有点到新加入的采样点的距离
        for j in range(N):
            dist = np.linalg.norm(points[j, :3] - points[farthest_point_index, :3])
            if dist < distances[j]:
                distances[j] = dist
    
    return sampled_points


def grouping_layer(sampled_points, all_points, R, K):
    '''
    Ball query，以FPS采样结果的每个点为球心，找到采样前总点云中距离每个球心最近的K个点（含球心），以便在后续步骤中提取局部特征。
    
    算法是：
        （1）预设搜索区域的半径R与子区域的点数K；
        （2）FPS采样函数已经提取出来了N'个点，以这N'个点为球心，画半径为R的球体（叫做query ball，也就是搜索区域）；
        （3）以每个样本点为球心，按照给定搜索半径R得到一个球形搜索区域，然后从该区域提取前K个最邻近点。如果搜索区域内点数不够，则重复采样；
        （4）得到N'个采样点各自对应的包含K个点的子区域，这是一个N'*K*（空间维度+特征维度）的点云。
    
    输入是：
        FPS采样前的点云、FPS采样后的点云、搜索半径R、子区域点数K。
    输出是：
        N'个采样点各自对应的包含K个点的子区域点云。
    '''
    # 读输入点云形状
    N_prime, D = sampled_points.shape
    N, _ = all_points.shape

    # 初始化返回的子区域点云数组
    local_regions = np.zeros((N_prime, K, D))

    for i in range(N_prime):
        center_point = sampled_points[i]
        
        # 计算所有点到球心的距离
        distances = np.linalg.norm(all_points[:, :3] - center_point[:3], axis=1)
        
        # 找到半径R内的点的索引
        within_radius_indices = np.where(distances <= R)[0]
        
        # 如果点的数量少于K个，则随机采样补充至K个
        if len(within_radius_indices) < K:
            additional_samples = np.random.choice(within_radius_indices, K - len(within_radius_indices), replace=True)
            nearest_indices = np.concatenate((within_radius_indices, additional_samples))
            nearest_indices = nearest_indices[np.argsort(distances[nearest_indices])]
        else:
            # 找到距离最近的K个点的索引
            nearest_indices = within_radius_indices[np.argsort(distances[within_radius_indices])[:K]]
        
        # 保存该球心的局部区域点云
        local_regions[i] = all_points[nearest_indices]
    
    return local_regions
    

def pointnet_layer(local_regions, mlp):
    '''
    PointNet卷积层，用卷积对采样点进行局部特征提取。
    
    算法是：
        多层感知机（MLP）* 1维卷积。
    
    输入是：
        Ball query得到的N'个采样点各自对应的包含K个点的子区域点云。
    输出是：
        包含新特征的点云（形状是N'*(3+新特征数量)）
    '''
    N,K,D = local_regions.shape
    reshape_points = local_regions.reshape(N, K * D)
    
    # 用1维卷积做特征提取
    # 1维卷积只能将同一点的不同特征做混合，不同点相对位置引起的特征是靠对点云的采样和分组实现的
    new_points = tf.convert_to_tensor(reshape_points, dtype=tf.float32)  # 确保输入是Tensor
    new_points = tf.expand_dims(new_points, axis=0)  # 在最外层增加一个batch维度以适应Conv1D
    
    for out_channel in mlp:
        new_points = layers.Conv1D(out_channel, 1, activation='relu')(new_points)
        new_points = tf.squeeze(new_points, axis=0)  # 去除多余的维度
        new_points = new_points.numpy()  # 将结果转换为NumPy数组以进行连接操作
        new_points = np.concatenate((reshape_points[:, :3], new_points), axis=1)
        new_points = tf.convert_to_tensor(new_points, dtype=tf.float32)  # 转换回Tensor以进行下一次卷积
        new_points = tf.expand_dims(new_points, axis=0)  # 再次增加一个维度以适应下一次Conv1D
    
    new_points = tf.squeeze(new_points, axis=0).numpy()  # 最后一次去除多余的维度并转换为NumPy数组
    
    return new_points


def SA_module(points, npoints, mlp):
    '''
    完整的Set Abstraction模块，由Sampling层、Grouping层、PointNet层串联组成。
    '''
    sampled_points = sampling_layer(points, npoints)
    local_regions = grouping_layer(sampled_points, points, R=0.5, K=5)
    new_points = pointnet_layer(local_regions, mlp)
    
    return new_points

In [9]:
# keras建模一定有batch维度，但之前的函数没有，需要修改

def build_pointnetplusplus(input_shape):
    '''
    PointNet++模型，需要调用SA模块
    输入为“单颗牙的3D模型在采样后的数据格式”，即num_samples*3
    输出为训练好的模型
    '''
    #输入层
    inputs = tf.keras.Input(shape=input_shape)
    l0_points = inputs
    
    # 三个SA模块
    l1_points = SA_module(l0_points, npoints=512, mlp=[64, 64, 128])
    l2_points = SA_module(l1_points, npoints=256, mlp=[128, 128, 256])
    l3_points = SA_module(l2_points, npoints=128, mlp=[256, 512, 1024])
    
    # 全局最大池化层，用于压缩数据量，也实现了数据扁平化
    x = layers.GlobalMaxPooling1D()(l3_points)
    
    # 全连接层和Dropout层，前者用于进行特征的整合和最终的输出，后者用于减少神经网络的过拟合
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # 输出层
    outputs = layers.Dense(1)(x)
    
    # 构建模型
    model = models.Model(inputs=inputs, outputs=outputs)
    
    '''
    编译模型：
    Adam优化器用于加快训练
    均方误差Mean Squared Error作为损失函数
    平均绝对误差Mean Absolute Error作为评价指标
    '''
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

In [10]:
def sample_points(points, num_samples):
    # 采样
    # 由于每颗牙的3D模型顶点数不同（约2100+），不方便直接用作神经网络的输入，需要统一输入的格式
    # 如果某颗牙的顶点数大于采样点数，则采样不可重复，否则采样可重复
    # 为了和vertices做变量名上的区分，这个函数中使用了points这个变量名，但实际上就是vertices的数据
    # 此处使用的是点云模型，如果用体素模型则有其他的配套手段
    if len(points) >= num_samples:
        sampled_points = points[np.random.choice(len(points), num_samples, replace=False)]
    else:
        sampled_points = points[np.random.choice(len(points), num_samples, replace=True)]
    return sampled_points


# 读序号和评分
'''此处为text文件地址，应当改为：’文件夹路径 + scores.txt'''
score_file_path = 'scores.txt'
indices, scores = read_scores(score_file_path)

# 读3D模型，并完成采样
'''设置采样点数为1024，可修改'''
num_samples = 1024  
Vertices = []
for i in indices:
    obj_file_path = f'{i}.obj'  # 此处为obj文件地址，应当改为：f’ + 文件夹路径 + {i}.obj
    points = read_obj_file(obj_file_path)
    sampled_points = sample_points(points, num_samples)
    Vertices.append(sampled_points)
Vertices = np.array(Vertices)
Scores = np.array(scores)

# 划分训练集和测试集
'''此处用的4：1随机分配，以后可以改成k折'''
Vertices_train, Vertices_test, Scores_train, Scores_test = train_test_split(Vertices, Scores, test_size=0.2, random_state=42)


# 训练
model = build_pointnetplusplus((num_samples, 3))
history = model.fit(Vertices_train, Scores_train, epochs=2, batch_size=8, validation_data=(Vertices_test, Scores_test))

NotImplementedError: Exception encountered when calling Lambda.call().

[1mWe could not automatically infer the shape of the Lambda's output. Please specify the `output_shape` argument for this Lambda layer.[0m

Arguments received by Lambda.call():
  • args=('<KerasTensor shape=(None, 1024, 3), dtype=float32, sparse=None, name=keras_tensor_1>',)
  • kwargs={'mask': 'None'}

In [ ]:
# 模型评价
loss, mae = model.evaluate(Vertices_test, Scores_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

In [ ]:
# 保存模型
model.save('saved_model.keras')

In [ ]:
# 加载模型
loaded_model = tf.keras.models.load_model('saved_model.keras')

In [ ]:
def evaluate(new_obj_file_path):
    # 读取并预处理新OBJ文件
    new_vertices = read_obj_file(new_obj_file_path)
    new_sampled_points = sample_points(new_vertices, num_samples)
    new_sampled_points = np.expand_dims(new_sampled_points, axis=0)  # 添加批次维度
    
    # 使用加载的模型进行预测
    predicted_score = loaded_model.predict(new_sampled_points)
    # print("Predicted Score:", predicted_score[0])
    
    return predicted_score

evaluate_result = {}
for i in range(150):
    new_obj_file_path = f'{i+1}.obj'  # 待评价对象的地址
    evaluate_result[i] = evaluate(new_obj_file_path)

In [ ]:
evaluate_result